In [106]:
!pip3 install pandas
import pandas as pd 


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [107]:
import json 


with open('trial_task.json', 'r') as file: 
    data = json.load(file)


Task 1      
Найти тариф стоимости доставки для каждого склада

In [108]:
from decimal import Decimal, ROUND_UP, DivisionByZero

shipping_rates = {}

for order in data:
    warehouse_name = order['warehouse_name']

    if warehouse_name in shipping_rates:
        continue

    product_count = 0
    
    for product in order['products']:
        product_count += product['quantity']

    try:
        shipping_rate = abs(Decimal(order['highway_cost']) / Decimal(product_count))\
            .quantize(Decimal("1.000"), rounding=ROUND_UP)
    except DivisionByZero:
        shipping_rate = None

    shipping_rates[warehouse_name] = shipping_rate

shipping_rates


{'Мордор': Decimal('10.000'),
 'хутор близ Диканьки': Decimal('15.000'),
 'отель Лето': Decimal('25.000'),
 'остров невезения': Decimal('5.000'),
 'гиперборея': Decimal('20.000')}

Task 2      
Найти суммарное количество , суммарный доход , суммарный расход и суммарную прибыль для каждого товара      
(представить как таблицу со столбцами 'product', 'quantity', 'income', 'expenses', 'profit')

In [109]:
products_df = pd.DataFrame(columns=['product', 'quantity', 'income', 'expenses', 'profit'])

for order in data:
    warehouse_name = order['warehouse_name']

    for product_info in order['products']:
        product_name = product_info['product']
        quantity = product_info['quantity']
        price = product_info['price']
        income = quantity * price
        expenses = shipping_rates[warehouse_name] * quantity
        profit = income - expenses

        products_df.loc[len(products_df)] = [product_name, quantity, income, expenses, profit]        

products_df = products_df.groupby('product').agg(
    quantity=pd.NamedAgg(column='quantity', aggfunc='sum'),
    income=pd.NamedAgg(column='income', aggfunc='sum'),
    expenses=pd.NamedAgg(column='expenses', aggfunc='sum'),
    profit=pd.NamedAgg(column='profit', aggfunc='sum')
).reset_index()

products_df

,product,quantity,income,expenses,profit
0,автограф Стаса Барецкого,48,28800,820.000,27980.000
1,билет в Израиль,58,58000,1050.000,56950.000
2,зеленая пластинка,61,610,920.000,-310.000
3,ломтик июльского неба,42,18900,670.000,18230.000
4,плюмбус,65,16250,940.000,15310.000
5,подписка на suppi-блог,33,4950,545.000,4405.000
6,статуэтка Ленина,68,13600,935.000,12665.000


Task 3      
Составить табличку со столбцами 'order_id' (id заказа) и 'order_profit'(прибыль полученная с заказа).        
А также вывести среднюю прибыль заказов

In [110]:
profit_df = pd.DataFrame(columns=['order_id', 'order_profit'])

for order in data:
    profit = 0    
    order_id = order['order_id']
    
    for product_info in order['products']:
        quantity = product_info['quantity']
        price = product_info['price']
        income = quantity * price
        profit += income
    
    profit += order['highway_cost']
    profit_df.loc[len(profit_df)] = [order_id, profit] 

average_profit = profit_df['order_profit'].mean()

print(profit_df)
print(f"average_profit:  {average_profit}")

    order_id  order_profit
0      11973          3980
1      62239           985
2      85794           -30
3      33684          1980
4      25824           975
..       ...           ...
95     28106           320
96     98100          1590
97     79293          1175
98      2930           720
99       124           705

[100 rows x 2 columns]
average_profit:  1352.3


Task 4            
Составить табличку типа 'warehouse_name' , 'product', 'quantity', 'profit',          
'percent_profit_product_of_warehouse' (процент прибыли продукта заказанного из определенного склада к прибыли этого склада)

In [111]:
warehouses_df = pd.DataFrame(columns=['warehouse_name', 'product', 'quantity', 'profit',])

for order in data:
    warehouse_name = order['warehouse_name']

    for product_info in order['products']:
        product_name = product_info['product']
        quantity = product_info['quantity']
        price = product_info['price']
        income = quantity * price
        expenses = shipping_rates[warehouse_name] * quantity
        profit = income - expenses

        warehouses_df.loc[len(warehouses_df)] = [warehouse_name, product_name, quantity, profit]        

warehouses_df = warehouses_df.groupby(['warehouse_name', 'product']).agg(
    quantity=pd.NamedAgg(column='quantity', aggfunc='sum'),
    profit=pd.NamedAgg(column='profit', aggfunc='sum')
).reset_index()

warehouse_profit = warehouses_df.groupby('warehouse_name')['profit'].sum()

percent_profit_product_of_warehouse = [round(i['profit'] / warehouse_profit[i['warehouse_name']], 3) for _, i in warehouses_df.iterrows()]

warehouses_df['percent_profit_product_of_warehouse'] = percent_profit_product_of_warehouse
warehouses_df['percent_profit_product_of_warehouse'] = warehouses_df['percent_profit_product_of_warehouse'].astype(float)

warehouses_df

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse
0,Мордор,автограф Стаса Барецкого,4,2360.000,0.138
1,Мордор,билет в Израиль,9,8910.000,0.523
2,Мордор,зеленая пластинка,11,0.000,0.000
3,Мордор,ломтик июльского неба,3,1320.000,0.077
4,Мордор,плюмбус,6,1440.000,0.084
5,Мордор,подписка на suppi-блог,8,1120.000,0.066
6,Мордор,статуэтка Ленина,10,1900.000,0.111
7,гиперборея,автограф Стаса Барецкого,12,6960.000,0.182
8,гиперборея,билет в Израиль,21,20580.000,0.538
9,гиперборея,зеленая пластинка,10,-100.000,-0.003


Task 5            
Взять предыдущую табличку и отсортировать 'percent_profit_product_of_warehouse' по убыванию,        
после посчитать накопленный процент.        
Накопленный процент - это новый столбец в этой табличке, который должен называться
'accumulated_percent_profit_product_of_warehouse'.      
По своей сути это постоянно растущая сумма отсортированного по убыванию столбца 'percent_profit_product_of_warehouse'.

In [112]:
sorted_warehouses_df = warehouses_df.sort_values(by=['warehouse_name', 'percent_profit_product_of_warehouse'], ascending=False)

sorted_warehouses_df['accumulated_percent_profit_product_of_warehouse'] = (
    sorted_warehouses_df.groupby('warehouse_name')['percent_profit_product_of_warehouse'].cumsum()
)

sorted_warehouses_df

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse
27,хутор близ Диканьки,автограф Стаса Барецкого,22,12870.000,0.346,0.346
28,хутор близ Диканьки,билет в Израиль,10,9850.000,0.265,0.611
31,хутор близ Диканьки,плюмбус,27,6345.000,0.170,0.781
33,хутор близ Диканьки,статуэтка Ленина,21,3885.000,0.104,0.885
30,хутор близ Диканьки,ломтик июльского неба,7,3045.000,0.082,0.967
32,хутор близ Диканьки,подписка на suppi-блог,10,1350.000,0.036,1.003
29,хутор близ Диканьки,зеленая пластинка,21,-105.000,-0.003,1.000
21,отель Лето,билет в Израиль,15,14625.000,0.541,0.541
20,отель Лето,автограф Стаса Барецкого,8,4600.000,0.170,0.711
23,отель Лето,ломтик июльского неба,9,3825.000,0.141,0.852


Task 6            
Присвоить A,B,C - категории на основании значения накопленного процента ('accumulated_percent_profit_product_of_warehouse').        
Если значение накопленного процента меньше или равно 70, то категория A.        
Если от 70 до 90 (включая 90), то категория Б. Остальное - категория C.         
Новый столбец обозначить в таблице как 'category'

In [113]:
sorted_warehouses_df['category'] = sorted_warehouses_df['accumulated_percent_profit_product_of_warehouse'].apply(
    lambda x: 'A' if x <= 0.7 else ('B' if 0.7 < x <= 0.9 else 'C')
)

sorted_warehouses_df

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse,category
27,хутор близ Диканьки,автограф Стаса Барецкого,22,12870.000,0.346,0.346,A
28,хутор близ Диканьки,билет в Израиль,10,9850.000,0.265,0.611,A
31,хутор близ Диканьки,плюмбус,27,6345.000,0.170,0.781,B
33,хутор близ Диканьки,статуэтка Ленина,21,3885.000,0.104,0.885,B
30,хутор близ Диканьки,ломтик июльского неба,7,3045.000,0.082,0.967,C
32,хутор близ Диканьки,подписка на suppi-блог,10,1350.000,0.036,1.003,C
29,хутор близ Диканьки,зеленая пластинка,21,-105.000,-0.003,1.000,C
21,отель Лето,билет в Израиль,15,14625.000,0.541,0.541,A
20,отель Лето,автограф Стаса Барецкого,8,4600.000,0.170,0.711,B
23,отель Лето,ломтик июльского неба,9,3825.000,0.141,0.852,B
